# Vectron Benchmark
## This notebook will go over all of Vectron's tests presented in the article. Note that some runtimes might differ from the ones presented in the article due to using a virtual container and the difference between the hardware configuratuons of the systems the simulation is run on. However, if the speedups are compared, the same results will be achieved.

## Also note that the input sequences for Codon, Vectron, C++ and CUDA (seqx.txt and seqy.txt) all have the same identical sequences, but due to the fact that C++ and CUDA handle pairs of sequences better, we prepared the same sequences fed to Codon and Vectron ready to be paired, called them with their appropriate names based on their number of sequences and fed them to C++ and CUDA

In [ ]:
import subprocess
import os
os.system("codon build /vectron/experiments_docker/source/vectron.codon")

In [ ]:
def compile(mode, src, vectron_path = ''):
    if mode == 'vectron':
        result = subprocess.run([vectron_path, '/codon-seq', '/vectron', f'{src}'], capture_output=True, text=True)
        print(result)
    elif mode == 'codon':
        result = subprocess.run(['codon', 'build', '-plugin', '/codon-seq', f'{src}', '-release'], capture_output=True, text=True)
        print(result)
    elif mode == 'cpp':             
        result = subprocess.run([
            'clang++', '-O3', '-msse4.2', '-funroll-loops', '-mfpmath=sse', '-march=native',
            f'{src}', '-o', f'{os.path.splitext(src)[0]}'
        ], capture_output=True, text=True)
        print(result)     
    elif mode == 'cuda':
        result = subprocess.run([
            'nvcc', '-o', f'{os.path.splitext(src)[0]}', f'{src}'
        ], capture_output=True, text=True)
        print(result)          

def exec(mode, src, ds_type):
    seq_x = ''
    seq_y = ''
    if ds_type == 'INT_small':
        if mode == 'vectron' or mode == 'codon':
            seq_x = '/vectron/experiments_docker/data/INT_small/seqx.txt'
            seq_y = '/vectron/experiments_docker/data/INT_small/seqy.txt'
        elif mode == 'cpp':
            seq_x = '/vectron/experiments_docker/data/CPP_small/seqx_4096.txt'
            seq_y = '/vectron/experiments_docker/data/CPP_small/seqy_4096.txt'  
    elif ds_type == 'INT_medium':
        if mode == 'vectron' or mode == 'codon':
            seq_x = '/vectron/experiments_docker/data/INT_medium/seqx.txt'
            seq_y = '/vectron/experiments_docker/data/INT_medium/seqy.txt'
        elif mode == 'cpp':
            seq_x = '/vectron/experiments_docker/data/CPP_medium/seqx_262144.txt'
            seq_y = '/vectron/experiments_docker/data/CPP_medium/seqy_262144.txt'  
    elif ds_type == 'INT_large':
        if mode == 'vectron' or mode == 'codon':
            seq_x = '/vectron/experiments_docker/data/INT_large/seqx.txt'
            seq_y = '/vectron/experiments_docker/data/INT_large/seqy.txt'
        elif mode == 'cpp':
            seq_x = '/vectron/experiments_docker/data/CPP_large/seqx_4194304.txt'
            seq_y = '/vectron/experiments_docker/data/CPP_large/seqy_4194304.txt'          

    if ds_type == 'FLOAT_small':
        if mode == 'vectron':
            seq_x = '/vectron/experiments_docker/data/FLOAT_small/seqx.txt'
            seq_y = '/vectron/experiments_docker/data/FLOAT_small/seqy.txt'
        elif mode == 'cuda' or mode == 'cpp':
            seq_x = '/vectron/experiments_docker/data/CUDA_small/seqx_256.txt'
            seq_y = '/vectron/experiments_docker/data/CUDA_small/seqy_256.txt'  
    elif ds_type == 'FLOAT_medium':
        if mode == 'vectron':
            seq_x = '/vectron/experiments_docker/data/FLOAT_medium/seqx.txt'
            seq_y = '/vectron/experiments_docker/data/FLOAT_medium/seqy.txt'
        elif mode == 'cuda' or mode == 'cpp':
            seq_x = '/vectron/experiments_docker/data/CUDA_medium/seqx_1024.txt'
            seq_y = '/vectron/experiments_docker/data/CUDA_medium/seqy_1024.txt'  
    elif ds_type == 'FLOAT_large':
        if mode == 'vectron':
            seq_x = '/vectron/experiments_docker/data/FLOAT_large/seqx.txt'
            seq_y = '/vectron/experiments_docker/data/FLOAT_large/seqy.txt'
        elif mode == 'cuda' or mode == 'cpp':
            seq_x = '/vectron/experiments_docker/data/CUDA_large/seqx_4096.txt'
            seq_y = '/vectron/experiments_docker/data/CUDA_large/seqy_4096.txt'          

    if mode == 'vectron':
        result = subprocess.run(f'{src} {seq_y} {seq_x} >vectron_out.txt', capture_output=True, text=True, shell=True)
        print(result.stderr)
    elif mode == 'codon':
        result = subprocess.run(f'{src} {seq_y} {seq_x} >codon_out.txt', capture_output=True, text=True, shell=True)
        print(result.stderr)
    elif mode == 'cpp':
        result = subprocess.run(f'{src} {seq_y} {seq_x} >cpp_out.txt', capture_output=True, text=True, shell=True)
        with open('cpp_out.txt', 'r') as file:
            lines = file.readlines()
            return(lines[-1].strip())
    elif mode == 'cuda':
        result = subprocess.run(f'{src} {seq_y} {seq_x} >cuda_out.txt', capture_output=True, text=True, shell=True)    
        with open('cpp_out.txt', 'r') as file:
            lines = file.readlines()
            return(lines[-1].strip())

### The following module will compile Vectron, Codon and C++ benchmarks on CPU in integer mode

In [ ]:
## COMPILING VECTRON EXPERIMENTS:
compile('vectron', '/vectron/experiments_docker/source/Vectron_INT/smith_waterman.codon', '/vectron/experiments_docker/source/vectron')
compile('vectron', '/vectron/experiments_docker/source/Vectron_INT/needleman_wunsch.codon', '/vectron/experiments_docker/source/vectron')
compile('vectron', '/vectron/experiments_docker/source/Vectron_INT/levenshtein_distance.codon', '/vectron/experiments_docker/source/vectron')
compile('vectron', '/vectron/experiments_docker/source/Vectron_INT/lcs.codon', '/vectron/experiments_docker/source/vectron')
compile('vectron', '/vectron/experiments_docker/source/Vectron_INT/hamming_distance.codon', '/vectron/experiments_docker/source/vectron')
compile('vectron', '/vectron/experiments_docker/source/Vectron_INT/manhattan_tourist.codon', '/vectron/experiments_docker/source/vectron')
compile('vectron', '/vectron/experiments_docker/source/Vectron_INT/min_cost_path.codon', '/vectron/experiments_docker/source/vectron')

## COMPILING CODON EXPERIMENTS:
compile('codon', '/vectron/experiments_docker/source/Codon/smith_waterman.codon')
compile('codon', '/vectron/experiments_docker/source/Codon/needleman_wunsch.codon')
compile('codon', '/vectron/experiments_docker/source/Codon/levenshtein_distance.codon')
compile('codon', '/vectron/experiments_docker/source/Codon/lcs.codon')
compile('codon', '/vectron/experiments_docker/source/Codon/hamming_distance.codon')
compile('codon', '/vectron/experiments_docker/source/Codon/manhattan_tourist.codon')
compile('codon', '/vectron/experiments_docker/source/Codon/min_cost_path.codon')

## COMPILING C++ EXPERIMENTS:
compile('cpp', '/vectron/experiments_docker/source/CPP/cpp_sw.cpp')
compile('cpp', '/vectron/experiments_docker/source/CPP/cpp_nmw.cpp')
compile('cpp', '/vectron/experiments_docker/source/CPP/cpp_LD.cpp')
compile('cpp', '/vectron/experiments_docker/source/CPP/cpp_lcs.cpp')
compile('cpp', '/vectron/experiments_docker/source/CPP/cpp_hd.cpp')
compile('cpp', '/vectron/experiments_docker/source/CPP/cpp_mt.cpp')
compile('cpp', '/vectron/experiments_docker/source/CPP/cpp_mcp.cpp')

### The following modules will execute Vectron, Codon and C++ respectively, and benchmark their runtimes for the small dataset (4096 sequence pairs)

In [ ]:
## RUNNING VECTRON EXPERIMENTS:
print("Smith Waterman Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/smith_waterman', 'INT_small')
print("Needleman Wunsch Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/needleman_wunsch', 'INT_small')
print("Levenshtein Distance Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/levenshtein_distance', 'INT_small')
print("Longest Common Subsequence Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/lcs', 'INT_small')
print("Hamming Distance Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/hamming_distance', 'INT_small')
print("Manhattan Tourist Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/manhattan_tourist', 'INT_small')
print("Minimum Cost Path Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/min_cost_path', 'INT_small')

In [ ]:
## RUNNING CODON EXPERIMENTS:
print("Smith Waterman Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/smith_waterman', 'INT_small')
print("Needleman Wunsch Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/needleman_wunsch', 'INT_small')
print("Levenshtein Distance Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/levenshtein_distance', 'INT_small')
print("Longest Common Subsequence Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/lcs', 'INT_small')
print("Hamming Distance Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/hamming_distance', 'INT_small')
print("Manhattan Tourist Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/manhattan_tourist', 'INT_small')
print("Minimum Cost Path Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/min_cost_path', 'INT_small')

In [ ]:
## RUNNING C++ EXPERIMENTS:
print("Smith Waterman Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_sw', 'INT_small')
print("Needleman Wunsch Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_nmw', 'INT_small')
print("Levenshtein Distance Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_LD', 'INT_small')
print("Longest Common Subsequence Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_lcs', 'INT_small')
print("Hamming Distance Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_hd', 'INT_small')
print("Manhattan Tourist Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_mt', 'INT_small')
print("Minimum Cost Path Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_mcp', 'INT_small')

### The following modules will execute Vectron, Codon and C++ respectively, and benchmark their runtimes for the medium dataset (262,144 sequence pairs)

In [ ]:
## RUNNING VECTRON EXPERIMENTS:
print("Smith Waterman Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/smith_waterman', 'INT_medium')
print("Needleman Wunsch Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/needleman_wunsch', 'INT_medium')
print("Levenshtein Distance Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/levenshtein_distance', 'INT_medium')
print("Longest Common Subsequence Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/lcs', 'INT_small')
print("Hamming Distance Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/hamming_distance', 'INT_medium')
print("Manhattan Tourist Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/manhattan_tourist', 'INT_medium')
print("Minimum Cost Path Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/min_cost_path', 'INT_medium')

In [ ]:
## RUNNING CODON EXPERIMENTS:
print("Smith Waterman Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/smith_waterman', 'INT_medium')
print("Needleman Wunsch Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/needleman_wunsch', 'INT_medium')
print("Levenshtein Distance Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/levenshtein_distance', 'INT_medium')
print("Longest Common Subsequence Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/lcs', 'INT_medium')
print("Hamming Distance Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/hamming_distance', 'INT_medium')
print("Manhattan Tourist Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/manhattan_tourist', 'INT_medium')
print("Minimum Cost Path Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/min_cost_path', 'INT_medium')

In [ ]:
## RUNNING C++ EXPERIMENTS:
print("Smith Waterman Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_sw', 'INT_medium')
print("Needleman Wunsch Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_nmw', 'INT_medium')
print("Levenshtein Distance Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_LD', 'INT_medium')
print("Longest Common Subsequence Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_lcs', 'INT_medium')
print("Hamming Distance Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_hd', 'INT_medium')
print("Manhattan Tourist Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_mt', 'INT_medium')
print("Minimum Cost Path Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_mcp', 'INT_medium')

### The following modules will execute Vectron, Codon and C++ respectively, and benchmark their runtimes for the large dataset (4,194,304 sequence pairs)

In [ ]:
## RUNNING VECTRON EXPERIMENTS:
print("Smith Waterman Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/smith_waterman', 'INT_large')
print("Needleman Wunsch Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/needleman_wunsch', 'INT_large')
print("Levenshtein Distance Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/levenshtein_distance', 'INT_large')
print("Longest Common Subsequence Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/lcs', 'INT_large')
print("Hamming Distance Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/hamming_distance', 'INT_large')
print("Manhattan Tourist Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/manhattan_tourist', 'INT_large')
print("Minimum Cost Path Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_INT/min_cost_path', 'INT_large')

In [ ]:
## RUNNING CODON EXPERIMENTS:
print("Smith Waterman Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/smith_waterman', 'INT_large')
print("Needleman Wunsch Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/needleman_wunsch', 'INT_large')
print("Levenshtein Distance Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/levenshtein_distance', 'INT_large')
print("Longest Common Subsequence Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/lcs', 'INT_large')
print("Hamming Distance Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/hamming_distance', 'INT_large')
print("Manhattan Tourist Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/manhattan_tourist', 'INT_large')
print("Minimum Cost Path Time:")
exec('codon', '/vectron/experiments_docker/source/Codon/min_cost_path', 'INT_large')

In [ ]:
## RUNNING C++ EXPERIMENTS:
print("Smith Waterman Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_sw', 'INT_large')
print("Needleman Wunsch Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_nmw', 'INT_large')
print("Levenshtein Distance Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_LD', 'INT_large')
print("Longest Common Subsequence Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_lcs', 'INT_large')
print("Hamming Distance Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_hd', 'INT_large')
print("Manhattan Tourist Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_mt', 'INT_large')
print("Minimum Cost Path Time:")
exec('cpp', '/vectron/experiments_docker/source/CPP/cpp_mcp', 'INT_large')

### The following module will compile Vectron, CUDA and C++ benchmarks on GPU in floating-point mode

In [ ]:
## COMPILING VECTRON EXPERIMENT:
compile('vectron', '/vectron/experiments_docker/source/Vectron_FLOAT/smith_waterman.codon', '/vectron/experiments_docker/source/vectron')

## COMPILING CUDA EXPERIMENT:
compile('cuda', '/vectron/experiments_docker/source/CUDA/cuda_sw.cu')

## COMPILING C++ EXPERIMENT:
compile('cpp', '/vectron/experiments_docker/source/CUDA/cpp_sw.cpp')

### The following module will execute Vectron, Codon and C++ respectively, and benchmark their runtimes for the small GPU dataset (256 sequence pairs)

In [ ]:
#RUNNING VECTRON EXPERIMENT:
print("Vectron Smith Waterman Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_FLOAT/smith_waterman', 'FLOAT_small')

#RUNNIG CUDA EXPERIMENT:
print("CUDA Smith Waterman Time:")
exec('cuda', '/vectron/experiments_docker/source/CUDA/cuda_sw', 'FLOAT_small')

#RUNNING C++ EXPERIMENT:
print("C++ Float Smith Waterman Time:")
exec('cpp', '/vectron/experiments_docker/source/CUDA/cpp_sw', 'FLOAT_small')

### The following module will execute Vectron, Codon and C++ respectively, and benchmark their runtimes for the medium GPU dataset (1024 sequence pairs)

In [ ]:
#RUNNING VECTRON EXPERIMENT:
print("Vectron Smith Waterman Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_FLOAT/smith_waterman', 'FLOAT_medium')

#RUNNIG CUDA EXPERIMENT:
print("CUDA Smith Waterman Time:")
exec('cuda', '/vectron/experiments_docker/source/CUDA/cuda_sw', 'FLOAT_medium')

#RUNNING C++ EXPERIMENT:
print("C++ Float Smith Waterman Time:")
exec('cpp', '/vectron/experiments_docker/source/CUDA/cpp_sw', 'FLOAT_medium')

### The following module will execute Vectron, Codon and C++ respectively, and benchmark their runtimes for the large GPU dataset (4096 sequence pairs)

In [ ]:
#RUNNING VECTRON EXPERIMENT:
print("Vectron Smith Waterman Time:")
exec('vectron', '/vectron/experiments_docker/source/Vectron_FLOAT/smith_waterman', 'FLOAT_large')

#RUNNIG CUDA EXPERIMENT:
print("CUDA Smith Waterman Time:")
exec('cuda', '/vectron/experiments_docker/source/CUDA/cuda_sw', 'FLOAT_large')

#RUNNING C++ EXPERIMENT:
print("C++ Float Smith Waterman Time:")
exec('cpp', '/vectron/experiments_docker/source/CUDA/cpp_sw', 'FLOAT_large')